In [275]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import os
import random
import warnings
from itertools import product

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from IPython import display
from tqdm.notebook import tqdm
from sklearn.exceptions import ConvergenceWarning

try:
    from myst_nb import glue
except ImportError:
    glue = lambda *args, **kwargs: _
# 一部の警告を無視
warnings.simplefilter("ignore", ConvergenceWarning)
warnings.simplefilter("ignore", FutureWarning)
os.environ["PYTHONWARNINGS"] = "ignore"

In [276]:
Xs=[]
Ys=[]
def load_data(data_name,N,M,K,pixel):
    for id in range(K):
        # id=random.randint(0,K-1)
        img=cv2.imread(f'./training_data/{data_name}/numbers_{id}.png')
        numbers=np.load(f'./training_data/{data_name}/numbers_{id}.npy')
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        # plt.imshow(img, cmap='gray')
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

        # plt.imshow(img, cmap='gray')
        # print(img.shape)
        img_rs=cv2.resize(gray,(pixel*M,pixel*N),interpolation=cv2.INTER_AREA)
    # img_rs=cv2.bitwise_not(img_rs)
    # print(img_rs.shape,i)

        for i in range(N):
            for j in range(M):
                digit_square = img_rs[i*pixel:(i+1)*pixel, j*pixel:(j+1)*pixel]
                x_i=digit_square.reshape(1, -1)
                # print(x_i.shape,i,j)
                y=numbers[i,j]
                Xs.append(x_i)
                Ys.append(y)

In [277]:
# 白地のまま学習する

# 画像を読み込む
try:
    train
except:
    train=1
save=1
pixel=20
N,M=50,50
K=2
# data_name='numbers'
# load_data(data_name,N,M,K,pixel)
load_data('numbers',50,50,7,pixel)
load_data('numbers_line_3',10,10,200,pixel)
load_data('numbers_line',50,50,8,pixel)
load_data('numbers_ln',10,10,150,pixel)
load_data('numbers_ln_m',10,10,200,pixel)
# load_data('numbers_line_wb',50,50,3,pixel)
load_data('numbers_line_wb_light',10,10,40,pixel)
load_data('numbers_wo0',50,50,3,pixel)
load_data('numbers_wo0_line',50,50,7,pixel)


        
Xs=pd.DataFrame(np.array(Xs).reshape(-1,pixel*pixel))
Ys=pd.DataFrame(np.array(Ys).reshape(-1,1),columns=['label'])
print(Xs.shape,Ys.shape)
df=pd.concat([Ys,Xs],axis=1)
# df=df.loc[df.iloc[:,0]!=0]
df.shape


(179000, 400) (179000, 1)


(179000, 401)

In [278]:
# i,j=10,5
# plt.imshow(img_rs[i*pixel:(i+1)*pixel, j*pixel:(j+1)*pixel], cmap='gray')


In [279]:
from sklearn import model_selection
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.ensemble import BaggingClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
X_org, y_org = df.iloc[:,1:], df.iloc[:,0]
X_org = np.array(X_org / 255.0, dtype="double")
y_org = np.array(y_org, dtype="uint8")
X, X_test, y, y_test = model_selection.train_test_split(
    X_org, y_org,test_size=0.2, shuffle=True,random_state=42
)
# use partial data 
# n_samples = 10000  
# X, y = X[:n_samples], y[:n_samples]
# X_test, y_test = X_test[:n_samples], y_test[:n_samples]

# スケーリングパラメータの計算
scaler = StandardScaler()
scaler.fit(X)

# データのスケーリング
X = scaler.transform(X)
X_test = scaler.transform(X_test)

# 訓練モデルの構築
lr_params = {"tol": 1.0e-3, "max_iter": 5000, "solver": "lbfgs"}
if train:
    # clf = LogisticRegression(**lr_params)
    clf = RandomForestClassifier(n_estimators=65,max_depth=13)
    # clf=MLPClassifier()
    # clf= KNeighborsClassifier(n_neighbors=20)
    # clf.fit(X, y)
    
    # 訓練モデルの構築 (弱識別器にロジスティック回帰を使用)
    # weak = LogisticRegression(**lr_params)
    # clf = BaggingClassifier(estimator=weak, n_estimators=10)
    # clf.fit(X, y)


    # clf = SVC(kernel="rbf", C=1.0, tol=1.0e-2, max_iter=-1)
    # O(n_samples^2 * n_features)
    # clf = SVC(kernel="linear", C=1.0, tol=1.0e-2, max_iter=-1)
    # clf.fit(X, y)
    # clf = LinearSVC(max_iter=50)
    # clf=GradientBoostingClassifier(n_estimators=10)
    # clf.fit(X, y)

    # # # # 訓練モデルの構築 (弱識別器にロジスティック回帰を使用)
    # weak = LogisticRegression(**lr_params)
    # clf = AdaBoostClassifier(estimator=weak, n_estimators=100)
    clf.fit(X, y)

    acc_train = 100.0 * clf.score(X, y)
    print("logis_acc_train", acc_train)

    # テストデータに対する識別精度の計算
    acc_test = 100.0 * clf.score(X_test, y_test)
    print("logis_acc_test", acc_test)

logis_acc_train 94.37569832402235
logis_acc_test 89.09217877094973


In [280]:
import sys
model_size = sys.getsizeof(clf)  # モデルオブジェクトのメモリ使用量を取得
print(f'Model size: {model_size} bytes')

Model size: 56 bytes


In [281]:
df

,label,0,1,2,3,4,5,6,7,8,...,390,391,392,393,394,395,396,397,398,399
0,8,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
1,1,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
2,2,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
3,1,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
4,5,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178995,7,57,170,170,170,170,170,170,170,170,...,255,255,255,255,255,255,255,255,255,255
178996,9,113,170,170,170,170,170,170,170,170,...,255,255,255,255,255,255,255,255,255,255
178997,9,113,170,170,170,170,170,170,170,170,...,255,255,255,255,255,255,255,255,255,170
178998,6,57,170,170,170,170,170,170,170,170,...,255,255,255,255,255,255,255,255,255,255


In [282]:
type(clf).__name__[:4]

'Rand'

In [283]:
import pickle
# # # save model and scaler
# # file="svc_rbf"
clf_name=type(clf).__name__[:4]
# print(clf_name)
# # train=1
data_name="numbers_mix_l2"
save=1
if train and save:
    with open(f'./pickle/{clf_name}_{data_name}_clf.pickle', mode='wb') as fp:
        pickle.dump(clf, fp)
    with open(f'./pickle/{clf_name}_{data_name}_scaler.pickle', mode='wb') as fp:
        pickle.dump(scaler, fp)

In [284]:

train=1
# reload
# scaler = pd.read_pickle('./pickle/rf_scaler.pickle')
# clf=pd.read_pickle('./pickle/rf_clf.pickle')
# X_test = scaler.transform(X_test)
acc_test = 100.0 * clf.score(X_test, y_test)
print("rf_acc_test", acc_test)

rf_acc_test 89.09217877094973


In [285]:
import recog_l2
# ディレクトリ内のファイルを取得する
dir_path = "./data/level1/"
files = os.listdir(dir_path)
# jpgファイルのみを取得する
jpg_files = [os.path.splitext(f)[0] for f in files if f.endswith('.jpg')]

# ファイル名を表示する
for f in jpg_files:
    print(f)
# jpg_files
# pixel=60
file="sample"
problem=recog_l2.recognize(path=f"./{dir_path}{file}.jpg",arc_epsilon=5e-2)
img=recog_l2.recognize(path=f"./{dir_path}{file}.jpg",arc_epsilon=5e-2,ret_img=1,erase_line=0)
# plt.imshow(img)
# problem

sample
sudoku_026
sudoku_027
sudoku_028
sudoku_029
sudoku_030


In [286]:
for file in jpg_files:
        # pixel=60
        problem=recog_l2.recognize(path=f"./{dir_path}{file}.jpg",pixel=pixel,clf=clf,scaler=scaler,n_close=1,n_open=2,prior_close=1,trim_percentage=0.008,mean_white_axis=0,arc_epsilon=5e-2,erase_line=0,otsu_times=1.2,clf_f_name="SVC")
        with open(f'./{dir_path}{file}.txt', 'r') as f:
            matrix_data = [[int(num) for num in line.split()] for line in f]
        # print matrix data by row
        # for row in matrix_data:
        #     print(row)
        matrix_array = np.array(matrix_data)
        validate=[matrix_data[i][j]==problem[i][j] for i in range(9) for j in range(9)]
        proba=sum(validate)/len(validate)
        zeros=np.sum(np.array(matrix_data) == 0)
        nonzeros=81-zeros
        failed=81-proba*81
        print(f"{file}","{:.4f}".format(proba),failed,f"failure rate: {failed/nonzeros:.2f}")
        # probas.append(proba)

sample 1.0000 0.0 failure rate: 0.00
sudoku_026 0.9877 1.0 failure rate: 0.02
sudoku_027 0.9877 1.0 failure rate: 0.02
sudoku_028 0.9630 3.0 failure rate: 0.07
sudoku_029 0.8765 10.0 failure rate: 0.24
sudoku_030 1.0000 0.0 failure rate: 0.00


In [287]:
import recog_l2
# ディレクトリ内のファイルを取得する
dir_path = "./data/level2/"
files = os.listdir(dir_path)
# jpgファイルのみを取得する
jpg_files = [os.path.splitext(f)[0] for f in files if f.endswith('.jpg')]

# ファイル名を表示する
for f in jpg_files:
    print(f)
# jpg_files
# pixel=60
file="sample"
problem=recog_l2.recognize(path=f"./{dir_path}{file}.jpg",arc_epsilon=5e-2)
img=recog_l2.recognize(path=f"./{dir_path}{file}.jpg",arc_epsilon=5e-2,ret_img=1,erase_line=0)
# plt.imshow(img)
# problem

sample
sudoku_00026
sudoku_00027
sudoku_00028
sudoku_00030
sudoku_00031


In [288]:
for file in jpg_files:
        # pixel=60
        problem=recog_l2.recognize(path=f"./{dir_path}{file}.jpg",pixel=pixel,clf=clf,scaler=scaler,n_close=1,n_open=2,prior_close=1,trim_percentage=0.008,mean_white_axis=0,arc_epsilon=5e-2,erase_line=0,otsu_times=1.2,clf_f_name="SVC")
        with open(f'./{dir_path}{file}.txt', 'r') as f:
            matrix_data = [[int(num) for num in line.split()] for line in f]
        # print matrix data by row
        # for row in matrix_data:
        #     print(row)
        matrix_array = np.array(matrix_data)
        validate=[matrix_data[i][j]==problem[i][j] for i in range(9) for j in range(9)]
        proba=sum(validate)/len(validate)
        zeros=np.sum(np.array(matrix_data) == 0)
        nonzeros=81-zeros
        failed=81-proba*81
        print(f"{file}","{:.4f}".format(proba),failed,f"failure rate: {failed/nonzeros:.2f}")
        # probas.append(proba)

sample 1.0000 0.0 failure rate: 0.00
sudoku_00026 1.0000 0.0 failure rate: 0.00
sudoku_00027 1.0000 0.0 failure rate: 0.00
sudoku_00028 1.0000 0.0 failure rate: 0.00
sudoku_00030 1.0000 0.0 failure rate: 0.00
sudoku_00031 1.0000 0.0 failure rate: 0.00
